In [1]:
from entity.models import Trip
trip = Trip.objects.all().prefetch_related('entity')[0]
trip
for td in trip.tripdata_set.all():
    pass
td

<TripData: Trip(1501000000000000002)|2015-01-16T00:05:39+00:00|97157>

In [7]:
trip.tripdata_set

<django.db.models.fields.related_descriptors.create_reverse_many_to_one_manager.<locals>.RelatedManager at 0x7ff7329ff470>

In [1]:
# Imports to help query the cube, these are used for the following sections.
from IPython.display import display, HTML
from itertools import islice
from django.db import connection

from datetime import datetime
from collections import namedtuple
from IPython.display import Markdown


cursor = connection.cursor()

num_results = 20  # Maximum results to print from a query
def format_results(results):
    if len(results) > 1:
        sep = '-:'
        nl = '\n'
        col_sep = '|'
    else:
        sep = ' = '
        nl = ''
        col_sep = ', '
    return Markdown(nl.join((
        col_sep.join([f for f in results[0]._fields]),  # Header row
        col_sep.join(['-:'] * len(results[0]._fields)) if len(results) > 1 else ' = ',  # Alignment Row
        '\n'.join([col_sep.join([str(getattr(r, f)) if getattr(r, f) is not None else 'ANY' for f in r._fields]) for r in results]).rstrip())
    ))

def execute(q_str, max_results=num_results):
    cursor.execute(q_str)
    nt_result = namedtuple('Result', [col[0] for col in cursor.description])
    return [nt_result(*r) for r in islice(cursor, None, max_results)]

In [2]:
l = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'A', 'B', 'C', 'D', 'E', 'F']
print(l[0:1:None])
print(l[-1:None:None])
print(l[::len(l)-1])

[0]
['F']
[0, 'F']


In [3]:
from entity.models import Trip, TripData

trip = Trip.objects.all()[0]
trip

<Trip: zhe305691|2011-12-19T23:16:02+00:00>

In [4]:
trip.organization

<Organization: Hangzhou>

In [5]:
qs = trip.tripdata_set.all()
qs

[<TripData: zhe305691|2011-12-19T23:16:02+00:00|1033679>, <TripData: zhe305691|2011-12-19T23:16:02+00:00|1>]

In [6]:
osm = qs[0].dataframe
rec = qs[1].dataframe

In [7]:
len(rec), len(osm)

(20, 18)

In [8]:
osm[0:1:None]  # First point

,osm
timestamp,
2011-12-20 07:17:44+08:00,1641423.0


In [9]:
osm[-1:None:None]  # Last point

,osm
timestamp,
2011-12-20 07:26:37+08:00,634288.0


In [10]:
osm[None:None:len(osm)-1]  # First and last point

,osm
timestamp,
2011-12-20 07:17:44+08:00,1641423.0
2011-12-20 07:26:37+08:00,634288.0


In [12]:
new_osm = osm.copy()
new_osm

,osm
timestamp,
2011-12-20 07:17:44+08:00,1641423.0
2011-12-20 07:18:05+08:00,1641423.0
2011-12-20 07:18:25+08:00,1641423.0
2011-12-20 07:18:46+08:00,1641423.0
2011-12-20 07:19:06+08:00,1641423.0
2011-12-20 07:19:47+08:00,1641423.0
2011-12-20 07:20:08+08:00,1641423.0
2011-12-20 07:20:28+08:00,1641423.0
2011-12-20 07:20:49+08:00,1641423.0


In [13]:
new_osm.insert(0, 'New_ID', range(1, 1 + len(new_osm)))
new_osm[:5]

,New_ID,osm
timestamp,,
2011-12-20 07:17:44+08:00,1,1641423.0
2011-12-20 07:18:05+08:00,2,1641423.0
2011-12-20 07:18:25+08:00,3,1641423.0
2011-12-20 07:18:46+08:00,4,1641423.0
2011-12-20 07:19:06+08:00,5,1641423.0


In [14]:
osm[:5]  # Show that new_osm is a copy.

,osm
timestamp,
2011-12-20 07:17:44+08:00,1641423.0
2011-12-20 07:18:05+08:00,1641423.0
2011-12-20 07:18:25+08:00,1641423.0
2011-12-20 07:18:46+08:00,1641423.0
2011-12-20 07:19:06+08:00,1641423.0


In [15]:
rec[:5]

,latitude,longitude,speed,heading,state
timestamp,,,,,
2011-12-20 07:16:02+08:00,30.380787,120.107710,39.63,260,0
2011-12-20 07:17:03+08:00,30.380617,120.101770,29.82,270,0
2011-12-20 07:17:44+08:00,30.377728,120.100130,19.26,190,0
2011-12-20 07:18:05+08:00,30.377623,120.100105,1.30,190,0
2011-12-20 07:18:25+08:00,30.377623,120.100105,1.30,190,0


In [16]:
rec['latitude']  # Returns a Series

timestamp
2011-12-20 07:16:02+08:00    30.380787
2011-12-20 07:17:03+08:00    30.380617
2011-12-20 07:17:44+08:00    30.377728
2011-12-20 07:18:05+08:00    30.377623
2011-12-20 07:18:25+08:00    30.377623
2011-12-20 07:18:46+08:00    30.376156
2011-12-20 07:19:06+08:00    30.373629
2011-12-20 07:19:47+08:00    30.369015
2011-12-20 07:20:08+08:00    30.367266
2011-12-20 07:20:28+08:00    30.365444
2011-12-20 07:20:49+08:00    30.363827
2011-12-20 07:21:30+08:00    30.362396
2011-12-20 07:21:50+08:00    30.361885
2011-12-20 07:24:34+08:00    30.359710
2011-12-20 07:24:55+08:00    30.358230
2011-12-20 07:25:15+08:00    30.357206
2011-12-20 07:25:36+08:00    30.356838
2011-12-20 07:25:56+08:00    30.356731
2011-12-20 07:26:17+08:00    30.356546
2011-12-20 07:26:37+08:00    30.355724
Name: latitude, dtype: float64

In [17]:
rec.index

DatetimeIndex(['2011-12-20 07:16:02+08:00', '2011-12-20 07:17:03+08:00',
               '2011-12-20 07:17:44+08:00', '2011-12-20 07:18:05+08:00',
               '2011-12-20 07:18:25+08:00', '2011-12-20 07:18:46+08:00',
               '2011-12-20 07:19:06+08:00', '2011-12-20 07:19:47+08:00',
               '2011-12-20 07:20:08+08:00', '2011-12-20 07:20:28+08:00',
               '2011-12-20 07:20:49+08:00', '2011-12-20 07:21:30+08:00',
               '2011-12-20 07:21:50+08:00', '2011-12-20 07:24:34+08:00',
               '2011-12-20 07:24:55+08:00', '2011-12-20 07:25:15+08:00',
               '2011-12-20 07:25:36+08:00', '2011-12-20 07:25:56+08:00',
               '2011-12-20 07:26:17+08:00', '2011-12-20 07:26:37+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', name='timestamp', freq=None)

In [18]:
osm.index

DatetimeIndex(['2011-12-20 07:17:44+08:00', '2011-12-20 07:18:05+08:00',
               '2011-12-20 07:18:25+08:00', '2011-12-20 07:18:46+08:00',
               '2011-12-20 07:19:06+08:00', '2011-12-20 07:19:47+08:00',
               '2011-12-20 07:20:08+08:00', '2011-12-20 07:20:28+08:00',
               '2011-12-20 07:20:49+08:00', '2011-12-20 07:21:30+08:00',
               '2011-12-20 07:21:50+08:00', '2011-12-20 07:24:34+08:00',
               '2011-12-20 07:24:55+08:00', '2011-12-20 07:25:15+08:00',
               '2011-12-20 07:25:36+08:00', '2011-12-20 07:25:56+08:00',
               '2011-12-20 07:26:17+08:00', '2011-12-20 07:26:37+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', name='timestamp', freq=None)

In [19]:
import pandas as pd
pd.concat([rec, osm], axis=1)[:5]

,latitude,longitude,speed,heading,state,osm
timestamp,,,,,,
2011-12-20 07:16:02+08:00,30.380787,120.107710,39.63,260,0,NaN
2011-12-20 07:17:03+08:00,30.380617,120.101770,29.82,270,0,NaN
2011-12-20 07:17:44+08:00,30.377728,120.100130,19.26,190,0,1641423.0
2011-12-20 07:18:05+08:00,30.377623,120.100105,1.30,190,0,1641423.0
2011-12-20 07:18:25+08:00,30.377623,120.100105,1.30,190,0,1641423.0


# Cube Time Partitioning

In [ ]:
import pytz

# Model value, Human readable
max_len = 0
for tz in pytz.all_timezones:
    max_len = max(len(tz), max_len)
max_len

In [13]:
from datetime import timedelta, datetime


def day_partitions_old(start_date, end_date, partition_size=1):
    """
    Returns a generator to get a tuple of start date and end date
    partitions that are partition_size.
    """
    units = 'days'  # TODO: Parameterize the units
    unit_range = getattr((end_date - start_date)/partition_size, units)
    partition_start = start_date
    for n in range(int(unit_range + 1)):
        partition_end = partition_start + timedelta(**{units: partition_size-1})
        yield partition_start, partition_end
        partition_start = partition_end + timedelta(**{units: 1})
        

def day_partitions(start_date, end_date, delta=None, **kwargs):
    """
    Returns a generator to get a tuple of start date and end date
    partitions that are partition_size.
    """
    if delta is None:
        delta = timedelta(**kwargs)
    unit_range = (end_date - start_date)/delta
    partition_start = start_date
    for n in range(int(unit_range + 1)):
        partition_end = partition_start + delta
        yield partition_start, partition_end
        partition_start = partition_end

In [20]:
start = datetime.now()
end = datetime.now() + timedelta(days=3)

In [21]:
list(day_partitions_old(start, end, partition_size=1))

[(datetime.datetime(2016, 6, 21, 21, 10, 1, 844999),
  datetime.datetime(2016, 6, 21, 21, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 22, 21, 10, 1, 844999),
  datetime.datetime(2016, 6, 22, 21, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 23, 21, 10, 1, 844999),
  datetime.datetime(2016, 6, 23, 21, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 24, 21, 10, 1, 844999),
  datetime.datetime(2016, 6, 24, 21, 10, 1, 844999))]

In [22]:
list(day_partitions(start, end, hours=10))

[(datetime.datetime(2016, 6, 21, 21, 10, 1, 844999),
  datetime.datetime(2016, 6, 22, 7, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 22, 7, 10, 1, 844999),
  datetime.datetime(2016, 6, 22, 17, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 22, 17, 10, 1, 844999),
  datetime.datetime(2016, 6, 23, 3, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 23, 3, 10, 1, 844999),
  datetime.datetime(2016, 6, 23, 13, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 23, 13, 10, 1, 844999),
  datetime.datetime(2016, 6, 23, 23, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 23, 23, 10, 1, 844999),
  datetime.datetime(2016, 6, 24, 9, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 24, 9, 10, 1, 844999),
  datetime.datetime(2016, 6, 24, 19, 10, 1, 844999)),
 (datetime.datetime(2016, 6, 24, 19, 10, 1, 844999),
  datetime.datetime(2016, 6, 25, 5, 10, 1, 844999))]

# Python Floating Point vs. Integer Division

In [20]:
82/16

5.125

In [21]:
82//16

5

# Return 2D Array of Column Name, Column Type

In [28]:
[str(t) for t in rec.dtypes]

['float64', 'float64', 'float64', 'int64', 'int64']